Notebook para corregir el fallo en la generacion de algunos archivos y graficas por colapso de la memoria del ordenador por la cantidad de puntos que hay

Los csv y png de resultados se generan a partir de los parquets, es cuestion de reprocesarlos si han colapsado una vez finalizados los entrenamientos

In [ ]:
import pandas as pd
import os
from Test_predictions import *
from custom_loss_functions import loss_function_name_list as all_loss_functions

# === Configuración ===
loss_function_name_list = all_loss_functions
horizons = [2,4]            
algorithms = ['Linear', 'LSTM', 'CNN']
#loss_function_name_list = ['cSigP06']
horizons = [4]            
algorithms = ['Linear']


# Rutas
test_results_path = "test_results"

# Valores de sensores
maximum_sensor_reading = 500
minimum_sensor_reading = 40

# Aquí deberías tener implementada la función
# def test_by_range(df, name_suffix, min_val, max_val): ...

# === Triple bucle ===
for loss_function_name in loss_function_name_list:  # For all loss functions -------------------------------------
    print(f"----- Loss function name: {loss_function_name} -----")
    for current_horizon in horizons:  # For all horizons -------------------------------------
        print(f"----- Prediction Horizon: {current_horizon} -----")
        for current_algorithm in algorithms:  # For all models ---------------------------------------
            print(f"----- Algorithm: {current_algorithm} -----")            
            parquet_file = os.path.join(
                test_results_path,
                f"df_test_results_vectors_{loss_function_name}_{current_algorithm}_H{current_horizon}.parquet"
            )
            
            if not os.path.exists(parquet_file):
                print(f"⚠️ Archivo no encontrado: {parquet_file}")
                continue

            print(f"\n📂 Leyendo: {parquet_file}")
            df_test_results = pd.read_parquet(parquet_file)

            print(f"Total test results for {current_algorithm} - Loss function {loss_function_name} "
                  f"- Prediction Horizon={current_horizon}: {len(df_test_results)} rows")

            try:
                df_metrics_by_range, df_zone_values, df_limits = test_by_range(
                    df_test_results,
                    f'_{loss_function_name}_{current_algorithm}_H{current_horizon}',
                    minimum_sensor_reading,
                    maximum_sensor_reading
                )

                # Guardar resultados
                df_metrics_by_range = df_metrics_by_range.round(2)
                df_metrics_by_range.to_csv(
                    f"{test_results_path}/metrics_performance_by_range_{loss_function_name}_{current_algorithm}_H{current_horizon}.csv",
                    index=False
                )

                df_zone_values.to_csv(
                    f"{test_results_path}/number_of_points_by_zones_{loss_function_name}_{current_algorithm}_H{current_horizon}.csv",
                    index=False
                )

                df_limits.to_csv(
                    f"{test_results_path}/predictions_out_of_limits_{loss_function_name}_{current_algorithm}_H{current_horizon}.csv",
                    index=False
                )

                print(f"✅ Procesado: {current_algorithm} | {loss_function_name} | H={current_horizon}")

            except Exception as e:
                print(f"❌ Error procesando {current_algorithm} | {loss_function_name} | H={current_horizon}")
                print(e)

print("\n🎉 Proceso completado")


----- Loss function name: cSigP06 -----
----- Prediction Horizon: 4 -----
----- Algorithm: Linear -----

📂 Leyendo: test_results/df_test_results_vectors_cSigP06_Linear_H4.parquet
Total test results for Linear - Loss function cSigP06 - Prediction Horizon=4: 19421406 rows
------- Zone: ENTIRE -------
Input Warning: the minimum predicted value (-565.40) is below the minimum input value (Inferior limit: 40 mg/dL).
Number of predicted values below the limit: 122272

Input Warning: the maximum predicted value (1001.95) is above the maximum input value (Superior limit: 500 mg/dL).
Number of predicted values above the limit: 1406

Zones: [A, B, C, D, E]
Number of points by zone: [13449728, 5310103, 129370, 485658, 46547]
Percentage of point by zone: [69.25, 27.34, 0.67, 2.5, 0.24]

Mean Squared Error (MSE): 1485.59
Root Mean Squared Error (RMSE): 38.54
Mean absolute Error (MAE): 27.08
Mean absolute percentage Error (MAPE): 0.18

-----------------------------------------------------------------

Comprobar archivos

In [ ]:
loss_function_name_list = all_loss_functions
horizons = [2,4]            
algorithms = ['Linear', 'LSTM', 'CNN']
test_results_path = "test_results"
plots_results_path = "plots"
folds = [1,2,3,4,5]  # Assuming 5 folds: 0 to 4
print ( "total", len(loss_function_name_list) * len(horizons) * len(algorithms) )#* (len(folds) + 1)
for loss_function_name in loss_function_name_list:  # For all loss functions -------------------------------------
    #print(f"----- Loss function name: {loss_function_name} -----")
    for current_horizon in horizons:  # For all horizons -------------------------------------
        #print(f"----- Prediction Horizon: {current_horizon} -----")
        for current_algorithm in algorithms:  # For all models ---------------------------------------
            #print(f"----- Algorithm: {current_algorithm} -----")            
            parquet_file = os.path.join(
                test_results_path,
                f"df_test_results_vectors_{loss_function_name}_{current_algorithm}_H{current_horizon}.parquet"
            )
            metrics_file = os.path.join(
                test_results_path,
                f"metrics_performance_by_range_{loss_function_name}_{current_algorithm}_H{current_horizon}.csv"
            )
            number_of_points_file = os.path.join(
                test_results_path,
                f"number_of_points_by_zones_{loss_function_name}_{current_algorithm}_H{current_horizon}.csv"
            )
            predictions_out_of_limits_file = os.path.join(
                test_results_path,
                f"predictions_out_of_limits_{loss_function_name}_{current_algorithm}_H{current_horizon}.csv"
            )
            plots_png_file = os.path.join(
                plots_results_path,
                f"Clarke_Error_Grid_{loss_function_name}_{current_algorithm}_H{current_horizon}.png"
            )
            '''
            plots_loss_pdf_file = os.path.join(
                plots_results_path,
                f"{loss_function_name}_{current_algorithm}_H{current_horizon}_Fold*_Loss.pdf"
            )
            plots_mse_pdf_file = os.path.join(
                plots_results_path,
                f"{loss_function_name}_{current_algorithm}_H{current_horizon}_MSE.pdf"
            )
            '''
            
            # --- Archivos por fold ---
            for fold in folds:
                df_test_results_fold_file = os.path.join(
                    test_results_path,
                    f"df_test_results_vectors_{loss_function_name}_{current_algorithm}_H{current_horizon}_Fold{fold}.parquet"
                )
                
                plots_loss_pdf_file = os.path.join(
                    plots_results_path,
                    f"{loss_function_name}_{current_algorithm}_H{current_horizon}_Fold{fold}_Loss.pdf"
                )
                plots_mse_pdf_file = os.path.join(
                    plots_results_path,
                    f"{loss_function_name}_{current_algorithm}_H{current_horizon}_Fold{fold}_MSE.pdf"
                )

                for file in [df_test_results_fold_file, plots_loss_pdf_file, plots_mse_pdf_file]:
                    if not os.path.exists(file):
                        print(f"⚠️ Archivo no encontrado: {file}")
            
            # Comprobar globales
            for file in [parquet_file, metrics_file, number_of_points_file, predictions_out_of_limits_file, plots_png_file]:
                if not os.path.exists(file):
                    print(f"⚠️ Archivo no encontrado: {file}")
       

total 42
